In [1]:
!pip install pennylane


[notice] A new release of pip available: 23.1.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import time
### DO NOT CHANGE ANYTHING BELOW THIS LINE

import pennylane as qml
from pennylane import numpy as np

WIRES = 2
LAYERS = 5
NUM_PARAMETERS = LAYERS * WIRES * 3

def variational_circuit(params,hamiltonian):
    """
    This is a template variational quantum circuit containing a fixed layout of gates with variable
    parameters. To be used as a QNode, it must either be wrapped with the @qml.qnode decorator or
    converted using the qml.QNode function.

    The output of this circuit is the expectation value of a Hamiltonian, somehow encoded in
    the hamiltonian argument

    Args:
        - params (np.ndarray): An array of optimizable parameters of shape (30,)
        - hamiltonian (np.ndarray): An array of real parameters encoding the Hamiltonian
        whose expectation value is returned.
    
    Returns:
        (float): The expectation value of the Hamiltonian
    """
    parameters = params.reshape((LAYERS, WIRES, 3))
    qml.templates.StronglyEntanglingLayers(parameters, wires=range(WIRES))
    return qml.expval(qml.Hermitian(hamiltonian, wires = [0,1]))

In [3]:
def optimize_circuit(hamiltonian, optimizer_name, stepsize):
    """Minimize the variational circuit and return its minimum value.
    You should create a device and convert the variational_circuit function 
    into an executable QNode. 
    Next, you should minimize the variational circuit using gradient-based 
    optimization to update the input params. 
    Return the optimized value of the QNode as a single floating-point number.

    Args:
        - params (np.ndarray): Input parameters to be optimized, of dimension 30
        - hamiltonian (np.ndarray): An array of real parameters encoding the Hamiltonian
        whose expectation value you should minimize.
    Returns:
        float: the value of the optimized QNode
    """
        
    hamiltonian = np.array(hamiltonian, requires_grad = False)

    hamiltonian = np.array(hamiltonian,float).reshape((2 ** WIRES), (2 ** WIRES))

    ### WRITE YOUR CODE BELOW THIS LINE
    
    ### Solution Template
    dev = qml.device('default.qubit', wires=WIRES) # Initialize the device.
    circuit = qml.QNode(variational_circuit, dev) # Instantiate the QNode from variational_circuit.

    def objective(params):
        return circuit(params, hamiltonian)

    # Minimize the circuit
    if optimizer_name == 'GRAD':
        opt = qml.GradientDescentOptimizer(stepsize=stepsize)
    elif optimizer_name == 'Momentum':
        opt = qml.MomentumOptimizer(stepsize=stepsize)
    elif optimizer_name == 'NesterovMomentum':
        opt = qml.NesterovMomentumOptimizer(stepsize=stepsize)
    elif optimizer_name == 'Adagrad':
        opt = qml.AdagradOptimizer(stepsize=stepsize)
    elif optimizer_name == 'RMSProp':
        opt = qml.RMSPropOptimizer(stepsize=stepsize)
    elif optimizer_name == 'Adam':
        opt = qml.AdamOptimizer(stepsize=stepsize)
    else:
        raise ValueError(f"Optimizer {optimizer_name} not recognized.")
        
    params = np.random.rand(NUM_PARAMETERS)
    
    start_time = time.time()
    steps = 100

    for _ in range(steps):
        params = opt.step(objective, params)

    # Otra forma de minimizar usando Scipy
    # from scipy.optimize import minimize
    # result = minimize(objective, params, method='BFGS')
    elapsed_time = time.time() - start_time
    return round(float(circuit(params, hamiltonian)), 8), elapsed_time

In [4]:
test_input = [0.863327072347624,0.0167108057202516,0.07991447085492759,0.0854049026262154, 0.0167108057202516,0.8237963773906136,-0.07695947154193797,0.03131548733285282, 0.07991447085492759,-0.07695947154193795,0.8355417021014687,-0.11345916130631205, 0.08540490262621539,0.03131548733285283,-0.11345916130631205,0.758156886827099]
optimizer_names = ['GRAD', 'Momentum', 'NesterovMomentum', 'Adagrad', 'RMSProp', 'Adam']
stepsizes = np.linspace(0.1, 1, 10)

target_result = 0.61745341

best_result = float('inf')
best_stepsize = None
best_optimizer = None
best_runtime = float('inf')

for stepsize in stepsizes:
    runtime_per_stepsize = []

    for optimizer_name in optimizer_names:
        result, elapsed_time = optimize_circuit(test_input, optimizer_name, stepsize)
        print(f"Optimizer: {optimizer_name}, Result: {result}, Elapsed Time: {elapsed_time} seconds")

        runtime_per_stepsize.append(elapsed_time)

        # Si el resultado es igual al valor objetivo y el tiempo de ejecución es menor, actualiza los mejores valores
        if result == target_result and elapsed_time < best_runtime:
            best_result = result
            best_stepsize = stepsize
            best_optimizer = optimizer_name
            best_runtime = elapsed_time

    print(f"Runtime for stepsize {stepsize}: {np.mean(runtime_per_stepsize)} seconds\n")

print(f"Best Optimizer: {best_optimizer}, Best Result: {best_result}, Best Stepsize: {best_stepsize}, Best Runtime: {best_runtime} seconds")

Optimizer: GRAD, Result: 0.62925957, Elapsed Time: 6.559036731719971 seconds
Optimizer: Momentum, Result: 0.61747441, Elapsed Time: 6.019376993179321 seconds
Optimizer: NesterovMomentum, Result: 0.61745345, Elapsed Time: 6.259066104888916 seconds
Optimizer: Adagrad, Result: 0.61745341, Elapsed Time: 6.079759359359741 seconds
Optimizer: RMSProp, Result: 0.62529663, Elapsed Time: 6.34599494934082 seconds
Optimizer: Adam, Result: 0.61745383, Elapsed Time: 6.29037070274353 seconds
Runtime for stepsize 0.1: 6.258934140205383 seconds

Optimizer: GRAD, Result: 0.61780125, Elapsed Time: 6.080897331237793 seconds
Optimizer: Momentum, Result: 0.61745511, Elapsed Time: 5.965817213058472 seconds
Optimizer: NesterovMomentum, Result: 0.6174536, Elapsed Time: 5.947551727294922 seconds
Optimizer: Adagrad, Result: 0.61745341, Elapsed Time: 6.021591424942017 seconds
Optimizer: RMSProp, Result: 0.62813938, Elapsed Time: 5.883145332336426 seconds
Optimizer: Adam, Result: 0.61745504, Elapsed Time: 6.047020